In [1]:
!pip install transformers

In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00


In [3]:
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.7 MB/s eta 0:00:00


In [4]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd drive/MyDrive/projects/compositional-reasoning-finetuning
%cd drive/MyDrive/projects/compositional-reasoning-finetuning

/content/drive/MyDrive/projects/compositional-reasoning-finetuning


## Chunk training data

In [ ]:
import json

model = "self_ask_train-answer_first=False-random_facts=True"
in_file = f"data/FinetuningData/{model}.json"
out_dir = f"data/FinetuningData/{model}/"
with open(in_file, "r") as f:
  train = json.load(f)

chunks = 15
chunk_size = len(train) // chunks
start = 0
end = chunk_size
chunk_num = 1
while end <= len(train):
  with open(out_dir+f"chunk-{chunk_num}.json", "w") as f:
    json.dump(train[start: end], f)
  print(start, end)
  start += chunk_size
  end += chunk_size
  chunk_num+=1

0 10325
10325 20650
20650 30975
30975 41300
41300 51625
51625 61950
61950 72275
72275 82600
82600 92925
92925 103250
103250 113575
113575 123900
123900 134225
134225 144550
144550 154875


## Self-Ask Finetuning

In [ ]:
from training_utils import finetune_self_ask

In [ ]:
# model_name
huggingface_model_name = 't5-small'
model = "self_ask_train-answer_first=False-random_facts=True"

# traing and validation file path
chunks = 15  # should match number of chunks in data directory
chunk_start = 13  # if you need to start at a different chunk number, change this
valid_file = 'data/FinetuningData/self_ask_dev-answer_first=False-random_facts=True.json'

checkpoint_dir = f'checkpoints/{model}/'
# prev_checkpoint = ""
prev_checkpoint = f"{checkpoint_dir}/weights-{chunk_start-1}.hdf5"

# hyper parameters
max_length = 300
batch_size = 32
epochs = 1

for chunk in range(chunk_start, chunks+1):
  print(chunk)
  train_file = f'data/FinetuningData/self_ask_train-answer_first=False-random_facts=True/chunk-{chunk}.json'

  # path and file name for checkpoint
  checkpoint_filepath = checkpoint_dir + f'weights-{chunk}' + '.{epoch:02d}-{batch:00005d}.hdf5'

  model_wrapper = finetune_self_ask(
      huggingface_model_name,
      train_file,
      valid_file,
      checkpoint_filepath,
      max_length,
      batch_size,
      epochs,
      prev_checkpoint)

  prev_checkpoint = f"{checkpoint_dir}/weights-{chunk}.hdf5"
  model_wrapper.save(prev_checkpoint, save_format="hdf5")
  del model_wrapper

13


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


322/322 [==============================] - 304s 851ms/step - loss: 0.1875 - accuracy: 0.9726 - val_loss: 0.0400 - val_accuracy: 0.9918


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14


...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


322/322 [==============================] - 312s 877ms/step - loss: 0.1811 - accuracy: 0.9730 - val_loss: 0.0406 - val_accuracy: 0.9918


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model_wrapper.save(f"models/{model}.h5", save_format="h5")

NameError: ignored